In [1]:
import sys
from warnings import warn
from itertools import chain

from azureml import core
from azureml.core import Workspace, Experiment, ScriptRunConfig
from azureml.core import Environment, Model
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

from azureml.core.runconfig import DockerConfiguration
from azureml.core.conda_dependencies import CondaDependencies

from rsidatasciencetools.azureutils.azurerun import AzureMLRun

In [2]:
from score import azure_configs

azure_config files found: ['/mnt/c/Users/NickRoseveare/Documents/Work/UbuntuVM/code/DataScienceTools/dstools/azureutils/example_run/azure_temp-run_train-on-amlcompute_1662741749/azure-config.json', '/mnt/c/Users/NickRoseveare/Documents/Work/UbuntuVM/code/DataScienceTools/dstools/azureutils/example_run/azure_temp-run_train-on-amlcompute_1662767070/azure-config.json', '/mnt/c/Users/NickRoseveare/Documents/Work/UbuntuVM/code/DataScienceTools/dstools/azureutils/example_run/azure_temp-run_train-on-amlcompute_1663103909/azure-config.json']
Performing interactive authentication. Please follow the instructions on the terminal.


KeyboardInterrupt: 

In [ ]:
arun = AzureMLRun('/home/nick/Work/code/DataScienceTools/dstools/azureutils/example_run/azure_temp-run_train-on-amlcompute_1662767070/',
                  # env_from_yaml_file=True,use_docker=True,
                  debug=1)
ws = arun.Workspace

In [ ]:
PYTHON_VERSION = "{major}.{minor}.{micro}".format(
    major=sys.version_info.major,
    minor=sys.version_info.minor,
    micro=sys.version_info.micro)
conda_dependencies = CondaDependencies.create(
    python_version=PYTHON_VERSION,
    conda_packages=(arun.conda_pkgs if len(arun.conda_pkgs) else None),
    pip_packages=(arun.pip_pkgs if len(arun.pip_pkgs) else None)
    )
conda_dependencies.__dict__

for whl in arun.whl_pkgs:
    whl_url = Environment.add_private_pip_wheel(workspace=arun._ws,
        file_path=whl,exist_ok=True)
    conda_dependencies.add_pip_package(whl_url)

In [ ]:

azure_env = Environment.get(ws, arun.azure_env_name)  #, version=None, label=None)
docker_matches = (azure_env.docker.base_image == (arun.use_docker
        if isinstance(arun.use_docker,str) else arun.default_base_docker_img))

conda_dep_check = azure_env.python.conda_dependencies.as_dict()
conda_dep_dict = conda_dependencies.as_dict()

conda_deps = set([dep for dep in conda_dep_check['dependencies'] 
    if not(isinstance(dep,dict))]), set([
        dep for dep in conda_dep_dict['dependencies']
            if not(isinstance(dep,dict))])
pip_deps = \
    set(chain.from_iterable(
        [dep['pip'] for dep in conda_dep_check['dependencies'] 
        if (isinstance(dep,dict) and ('pip' in dep))])), \
    set(chain.from_iterable(
        [dep['pip'] for dep in conda_dep_dict['dependencies'] 
            if (isinstance(dep,dict) and ('pip' in dep))]))

In [ ]:
docker_matches, (conda_dep_dict['name'], conda_dep_check['name']), (
    conda_deps[0].symmetric_difference(conda_deps[1]),
        pip_deps[0].symmetric_difference(pip_deps[1]))

In [ ]:
if docker_matches and (conda_dep_dict['name'] == conda_dep_check['name'] and
        (len(conda_deps[0].symmetric_difference(conda_deps[1])) == 0) and
        (len(pip_deps[0].symmetric_difference(pip_deps[1])) == 0)):
    found_matching = True
    # arun.azure_env = azure_env
    # arun.env_status = RegistrationStatus.REGISTERED_EXISTING
    warn(f'found matching environement to use: {arun.azure_env_name}')
    